# 3. Face Emotion Training - ResNet50 on FER2013

Train ResNet50 for facial emotion recognition. Run in Google Colab with GPU.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

print(f'TensorFlow: {tf.__version__}')
print(f'GPU: {tf.config.list_physical_devices("GPU")}')

In [ ]:
# FER2013 classes
EMOTION_LABELS = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
NUM_CLASSES = 7
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_PATH = '../data/face_data'  # Should have train/ and test/ subfolders

In [ ]:
# Data generators with augmentation
# FER2013 is 48x48 grayscale - we resize to 224x224 and convert to RGB

def preprocess_fer(img):
    # Grayscale to RGB: repeat channel 3 times
    if len(img.shape) == 2:
        img = np.stack([img]*3, axis=-1)
    elif img.shape[-1] == 1:
        img = np.concatenate([img]*3, axis=-1)
    return img

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load data
train_data = train_gen.flow_from_directory(
    os.path.join(DATA_PATH, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    color_mode='rgb'
)

test_data = test_gen.flow_from_directory(
    os.path.join(DATA_PATH, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    color_mode='rgb'
)

print(f'Train samples: {train_data.samples}')
print(f'Test samples: {test_data.samples}')

In [ ]:
# Build ResNet50 model
base = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze base layers initially
for layer in base.layers:
    layer.trainable = False

# Add custom head
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base.input, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Callbacks
callbacks = [
    ModelCheckpoint('../models/resnet_face/resnet50_emotion.h5', save_best_only=True, monitor='val_accuracy'),
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

os.makedirs('../models/resnet_face', exist_ok=True)

In [ ]:
# Phase 1: Train head only
print('Phase 1: Training head layers...')
history1 = model.fit(train_data, epochs=10, validation_data=test_data, callbacks=callbacks)

In [ ]:
# Phase 2: Fine-tune top layers
print('Phase 2: Fine-tuning...')
for layer in base.layers[-30:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history2 = model.fit(train_data, epochs=20, validation_data=test_data, callbacks=callbacks)

In [ ]:
# Evaluate
loss, acc = model.evaluate(test_data)
print(f'Test Accuracy: {acc:.4f}')

# Save final model
model.save('../models/resnet_face/resnet50_emotion_final.h5')
print('Model saved!')